In [41]:
#######################################################################################################################################################################
# Angle Calculation using Mediapipe Pose Visualization #
#######################################################################################################################################################################
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Function to calculate the angle between three points
def calculate_angle(a, b, c):
    """Calculate angle between three 2D points using arctangent function."""
    ba = np.array(a) - np.array(b)
    bc = np.array(c) - np.array(b)

    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))

    return np.degrees(angle)

# Open webcam
cap = cv2.VideoCapture("asitwas.webm")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to RGB
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark

        # Define key joints (Using Mediapipe landmark indices)
        joint_ids = {
            "nose": 0,
            "left_shoulder": 11, "right_shoulder": 12,
            "left_elbow": 13, "right_elbow": 14,
            "left_wrist": 15, "right_wrist": 16,
            "left_waist": 23, "right_waist": 24  # Waist is hip landmarks
        }

        # Extract 2D coordinates
        joints = {}
        for name, idx in joint_ids.items():
            if landmarks[idx].visibility > 0.5:  # Ensure the joint is visible
                joints[name] = (int(landmarks[idx].x * frame.shape[1]), 
                                int(landmarks[idx].y * frame.shape[0]))

        # Compute mid_shoulder (midpoint of left and right shoulder)
        if all(k in joints for k in ["left_shoulder", "right_shoulder"]):
            joints["mid_shoulder"] = ((joints["left_shoulder"][0] + joints["right_shoulder"][0]) // 2,
                                      (joints["left_shoulder"][1] + joints["right_shoulder"][1]) // 2)

        # Compute angles only if required points exist
        angles = {}
        if all(k in joints for k in ["left_shoulder", "left_elbow", "left_wrist"]):
            angles["Left Elbow"] = calculate_angle(joints["left_shoulder"], joints["left_elbow"], joints["left_wrist"])
        if all(k in joints for k in ["right_shoulder", "right_elbow", "right_wrist"]):
            angles["Right Elbow"] = calculate_angle(joints["right_shoulder"], joints["right_elbow"], joints["right_wrist"])
        if all(k in joints for k in ["left_elbow", "left_shoulder", "left_waist"]):
            angles["Left Arm Lift"] = calculate_angle(joints["left_elbow"], joints["left_shoulder"], joints["left_waist"])
        if all(k in joints for k in ["right_elbow", "right_shoulder", "right_waist"]):
            angles["Right Arm Lift"] = calculate_angle(joints["right_elbow"], joints["right_shoulder"], joints["right_waist"])
        if all(k in joints for k in ["nose", "mid_shoulder", "left_shoulder"]):
            angles["Neck Angle"] = calculate_angle(joints["nose"], joints["mid_shoulder"], joints["left_shoulder"])

        # Draw key points and lines
        for (p1, p2, p3, label, text_pos) in [
            ("left_shoulder", "left_elbow", "left_wrist", "Left Elbow", "left_elbow"),
            ("right_shoulder", "right_elbow", "right_wrist", "Right Elbow", "right_elbow"),
            ("left_elbow", "left_shoulder", "left_waist", "Left Arm Lift", "left_shoulder"),
            ("right_elbow", "right_shoulder", "right_waist", "Right Arm Lift", "right_shoulder")
        ]:
            if all(k in joints for k in [p1, p2, p3]):
                # Draw lines
                cv2.line(frame, joints[p1], joints[p2], (0, 255, 0), 3)
                cv2.line(frame, joints[p2], joints[p3], (0, 255, 0), 3)

                # Draw key points
                cv2.circle(frame, joints[p1], 6, (0, 0, 255), -1)
                cv2.circle(frame, joints[p2], 6, (255, 0, 0), -1)
                cv2.circle(frame, joints[p3], 6, (0, 255, 255), -1)

                # Display angle
                if label in angles:
                    angle_text = f"{label}: {int(angles[label])}"
                    cv2.putText(frame, angle_text, (joints[text_pos][0] + 20, joints[text_pos][1]), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

        # Draw neck angle separately
        if all(k in joints for k in ["nose", "mid_shoulder", "left_shoulder"]) and "Neck Angle" in angles:
            cv2.line(frame, joints["nose"], joints["mid_shoulder"], (0, 255, 0), 3)
            cv2.line(frame, joints["mid_shoulder"], joints["left_shoulder"], (0, 255, 0), 3)

            cv2.circle(frame, joints["nose"], 6, (0, 0, 255), -1)
            cv2.circle(frame, joints["mid_shoulder"], 6, (255, 0, 0), -1)
            cv2.circle(frame, joints["left_shoulder"], 6, (0, 255, 255), -1)

            # Display neck angle
            angle_text = f"Neck Angle: {int(angles['Neck Angle'])}"
            cv2.putText(frame, angle_text, (joints["nose"][0] + 20, joints["nose"][1]), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

    # Show the frame
    cv2.imshow("Pose Angle Visualization", frame)

    # Exit if 'q' is pressed
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


c:\Users\kshit\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [42]:
#######################################################################################################################################################################
# Move Detection, Recording and Preprocessing #
#######################################################################################################################################################################

import cv2
import mediapipe as mp
import json
import numpy as np
import time
import os

# Load timeline.json
with open("timeline.json", "r") as f:
    timeline_data = json.load(f)

# Extract moves
moves = timeline_data["moves"]

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Directory to save processed reference data
output_dir = "reference_moves"
os.makedirs(output_dir, exist_ok=True)

def calculate_angle(a, b, c):
    """Calculate angle between three 3D points using the arctangent function."""
    ba = np.array(a) - np.array(b)
    bc = np.array(c) - np.array(b)

    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    
    return np.degrees(angle)

def process_frame(image):
    """Extract pose landmarks from a frame and compute angles."""
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)
    
    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark
        
        # Extract key joint coordinates (normalized values)
        joint_ids = {
            "nose": 0,
            "left_shoulder": 11, "right_shoulder": 12,
            "left_elbow": 13, "right_elbow": 14,
            "left_wrist": 15, "right_wrist": 16,
            "left_hip": 23, "right_hip": 24
        }
        
        joints = {name: (landmarks[idx].x, landmarks[idx].y, landmarks[idx].z) for name, idx in joint_ids.items()}

        # Compute mid-shoulder point
        if "left_shoulder" in joints and "right_shoulder" in joints:
            mid_shoulder = (
                (joints["left_shoulder"][0] + joints["right_shoulder"][0]) / 2,
                (joints["left_shoulder"][1] + joints["right_shoulder"][1]) / 2,
                (joints["left_shoulder"][2] + joints["right_shoulder"][2]) / 2
            )
        else:
            mid_shoulder = None

        # Compute angles
        angles = {}

        if all(k in joints for k in ["left_shoulder", "left_elbow", "left_wrist"]):
            angles["left_elbow"] = calculate_angle(joints["left_shoulder"], joints["left_elbow"], joints["left_wrist"])
        if all(k in joints for k in ["right_shoulder", "right_elbow", "right_wrist"]):
            angles["right_elbow"] = calculate_angle(joints["right_shoulder"], joints["right_elbow"], joints["right_wrist"])
        if all(k in joints for k in ["left_elbow", "left_shoulder", "left_hip"]):
            angles["left_arm_lift"] = calculate_angle(joints["left_elbow"], joints["left_shoulder"], joints["left_hip"])
        if all(k in joints for k in ["right_elbow", "right_shoulder", "right_hip"]):
            angles["right_arm_lift"] = calculate_angle(joints["right_elbow"], joints["right_shoulder"], joints["right_hip"])
        if mid_shoulder and "nose" in joints:
            angles["neck_angle"] = calculate_angle(joints["nose"], mid_shoulder, joints["left_shoulder"])

        return angles
    
    return None

# Open webcam
cap = cv2.VideoCapture(0)

for move in moves:
    move_name = move["name"]
    duration = move["duration"]
    
    print(f"Get ready for move: {move_name}")
    time.sleep(2)  # Give user time to prepare
    
    print(f"Recording move: {move_name} (duration: {duration}s)...")
    
    start_time = time.time()
    capture_time = start_time + (duration / 2)  # Capture a frame at the middle of the move
    
    captured_frame = None

    while time.time() - start_time < duration:
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture frame!")
            continue
        
        if time.time() >= capture_time and captured_frame is None:
            captured_frame = frame.copy()
    
    if captured_frame is not None:
        angles = process_frame(captured_frame)
        if angles:
            # Save angles for this move
            with open(os.path.join(output_dir, f"{move_name}.json"), "w") as f:
                json.dump(angles, f, indent=4)
            print(f"Saved reference angles for {move_name}")
        else:
            print(f"Failed to detect pose for {move_name}")

cap.release()
cv2.destroyAllWindows()

Get ready for move: asitwas_jacket


KeyboardInterrupt: 

In [ ]:
## 1 Frame Data Recorded for each move ##

import cv2
import mediapipe as mp
import json
import numpy as np
import time
import os
import joblib

# Load timeline.json
with open("timeline.json", "r") as f:
    timeline_data = json.load(f)

# Extract moves
moves = timeline_data["moves"]

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Directory to save processed reference data
output_dir = "reference_moves"
os.makedirs(output_dir, exist_ok=True)

# Load the dance video
video_path = "asitwas.webm"  # Change this to your video file
cap = cv2.VideoCapture(video_path)

# Check if video loaded successfully
if not cap.isOpened():
    print("Error: Could not open video file.")
    exit()

fps = cap.get(cv2.CAP_PROP_FPS)  # Frames per second
print(f"Video FPS: {fps}")

def calculate_angle(a, b, c):
    """Calculate angle between three 3D points using the arctangent function."""
    ba = np.array(a) - np.array(b)
    bc = np.array(c) - np.array(b)

    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))

    return np.degrees(angle)

def process_frame(image):
    """Extract pose landmarks from a frame and compute angles."""
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark

        # Extract key joint coordinates (normalized values)
        joint_ids = {
            "nose": 0,
            "left_shoulder": 11, "right_shoulder": 12,
            "left_elbow": 13, "right_elbow": 14,
            "left_wrist": 15, "right_wrist": 16,
            "left_hip": 23, "right_hip": 24
        }

        joints = {name: (landmarks[idx].x, landmarks[idx].y, landmarks[idx].z) for name, idx in joint_ids.items()}

        # Compute mid-shoulder point
        if "left_shoulder" in joints and "right_shoulder" in joints:
            mid_shoulder = (
                (joints["left_shoulder"][0] + joints["right_shoulder"][0]) / 2,
                (joints["left_shoulder"][1] + joints["right_shoulder"][1]) / 2,
                (joints["left_shoulder"][2] + joints["right_shoulder"][2]) / 2
            )
        else:
            mid_shoulder = None

        # Compute angles
        angles = {}

        if all(k in joints for k in ["left_shoulder", "left_elbow", "left_wrist"]):
            angles["left_elbow"] = calculate_angle(joints["left_shoulder"], joints["left_elbow"], joints["left_wrist"])
        if all(k in joints for k in ["right_shoulder", "right_elbow", "right_wrist"]):
            angles["right_elbow"] = calculate_angle(joints["right_shoulder"], joints["right_elbow"], joints["right_wrist"])
        if all(k in joints for k in ["left_elbow", "left_shoulder", "left_hip"]):
            angles["left_arm_lift"] = calculate_angle(joints["left_elbow"], joints["left_shoulder"], joints["left_hip"])
        if all(k in joints for k in ["right_elbow", "right_shoulder", "right_hip"]):
            angles["right_arm_lift"] = calculate_angle(joints["right_elbow"], joints["right_shoulder"], joints["right_hip"])
        if mid_shoulder and "nose" in joints:
            angles["neck_angle"] = calculate_angle(joints["nose"], mid_shoulder, joints["left_shoulder"])

        return {"joints": joints, "angles": angles}

    return None

# Process each move in the timeline
reference_data = {}

for move in moves:
    move_name = move["name"]
    start_time = move["time"] + 14.828
    duration = move["duration"]

    mid_time = start_time + (duration / 2)  # Middle of the move

    frame_idx = int(mid_time * fps)  # Convert to frame index
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)  # Seek to the frame

    ret, frame = cap.read()
    if not ret:
        print(f"Failed to capture frame for {move_name} at time {mid_time}s (frame {frame_idx})!")
        continue

    print(f"Processing {move_name} at frame {frame_idx}...")

    pose_data = process_frame(frame)
    if pose_data:
        reference_data[move_name] = pose_data
        print(f"✔ Saved pose data for {move_name}")
    else:
        print(f"❌ Failed to detect pose for {move_name}")

# Save the reference data using joblib
joblib.dump(reference_data, os.path.join(output_dir, "reference_data.pkl"))
print(f"✅ Reference data saved to {output_dir}/reference_data.pkl")

cap.release()
cv2.destroyAllWindows()


Video FPS: 25.0
Processing asitwas_jacket at frame 495...
✔ Saved pose data for asitwas_jacket
Processing asitwas_snap at frame 513...
✔ Saved pose data for asitwas_snap
Processing asitwas_arm_up at frame 532...
✔ Saved pose data for asitwas_arm_up
Processing asitwas_snap_down_start at frame 590...
✔ Saved pose data for asitwas_snap_down_start
Processing asitwas_snap_down at frame 607...
✔ Saved pose data for asitwas_snap_down
Processing asitwas_snap_down at frame 625...
✔ Saved pose data for asitwas_snap_down
Processing asitwas_snap_down at frame 642...
✔ Saved pose data for asitwas_snap_down
Processing asitwas_snap_down_start_2 at frame 659...
✔ Saved pose data for asitwas_snap_down_start_2
Processing asitwas_snap_down at frame 676...
✔ Saved pose data for asitwas_snap_down
Processing asitwas_snap_down at frame 694...
✔ Saved pose data for asitwas_snap_down
Processing asitwas_snap_down_end at frame 711...
✔ Saved pose data for asitwas_snap_down_end
Processing asitwas_arm_down at fram

In [45]:
## 3 Frame Data Recorded for each move (just left hand)##

import cv2
import mediapipe as mp
import json
import numpy as np
import os
import joblib

# Load timeline.json
with open("timeline.json", "r") as f:
    timeline_data = json.load(f)

# Extract moves
moves = timeline_data["moves"]

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Directory to save processed reference data
output_dir = "reference_moves_three"
os.makedirs(output_dir, exist_ok=True)

# Load the dance video
video_path = "asitwas.webm"  # Change this to your video file
cap = cv2.VideoCapture(video_path)

# Check if video loaded successfully
if not cap.isOpened():
    print("Error: Could not open video file.")
    exit()

fps = cap.get(cv2.CAP_PROP_FPS)  # Frames per second
print(f"Video FPS: {fps}")

def calculate_angle(a, b, c):
    """Calculate angle between three 3D points using the arctangent function."""
    ba = np.array(a) - np.array(b)
    bc = np.array(c) - np.array(b)

    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))

    return np.degrees(angle)

def process_frame(image):
    """Extract pose landmarks from a frame and compute only left elbow and left arm lift angles."""
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark

        # Extract key joint coordinates (normalized values)
        joint_ids = {
            "left_shoulder": 11,
            "left_elbow": 13,
            "left_wrist": 15,
            "left_hip": 23
        }

        joints = {name: (landmarks[idx].x, landmarks[idx].y, landmarks[idx].z) for name, idx in joint_ids.items()}

        # Compute angles
        angles = {}

        if all(k in joints for k in ["left_shoulder", "left_elbow", "left_wrist"]):
            angles["left_elbow"] = calculate_angle(joints["left_shoulder"], joints["left_elbow"], joints["left_wrist"])
        if all(k in joints for k in ["left_elbow", "left_shoulder", "left_hip"]):
            angles["left_arm_lift"] = calculate_angle(joints["left_elbow"], joints["left_shoulder"], joints["left_hip"])

        return {"joints": joints, "angles": angles}

    return None



# Process each move in the timeline
reference_data = {}

for move in moves:
    move_name = move["name"]
    start_time = move["time"] + 14.828  # Adjust offset if needed
    duration = move["duration"]

    # Get three frames: start, middle, end
    frame_times = [
        start_time,  # Start frame
        start_time + (duration / 2),  # Middle frame
        start_time + duration  # End frame
    ]

    move_data = []  # Store all frames for this move

    for time_point in frame_times:
        frame_idx = int(time_point * fps)  # Convert to frame index
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)  # Seek to the frame

        ret, frame = cap.read()
        if not ret:
            print(f"❌ Failed to capture frame for {move_name} at {time_point}s (frame {frame_idx})")
            continue

        print(f"Processing {move_name} at frame {frame_idx}...")

        pose_data = process_frame(frame)
        if pose_data:
            move_data.append(pose_data)
        else:
            print(f"❌ Failed to detect pose for {move_name} at frame {frame_idx}")

    if move_data:
        reference_data[move_name] = move_data
        print(f"✔ Saved {len(move_data)} frames for {move_name}")
    else:
        print(f"❌ No valid pose data for {move_name}")

# Save the reference data using joblib
joblib.dump(reference_data, os.path.join(output_dir, "reference_data.pkl"))
print(f"✅ Reference data saved to {output_dir}/reference_data.pkl")

cap.release()
cv2.destroyAllWindows()


Video FPS: 25.0
Processing asitwas_jacket at frame 487...
Processing asitwas_jacket at frame 495...
Processing asitwas_jacket at frame 504...
✔ Saved 3 frames for asitwas_jacket
Processing asitwas_snap at frame 504...
Processing asitwas_snap at frame 513...
Processing asitwas_snap at frame 521...
✔ Saved 3 frames for asitwas_snap
Processing asitwas_arm_up at frame 521...
Processing asitwas_arm_up at frame 532...
Processing asitwas_arm_up at frame 543...
✔ Saved 3 frames for asitwas_arm_up
Processing asitwas_snap_down_start at frame 581...
Processing asitwas_snap_down_start at frame 590...
Processing asitwas_snap_down_start at frame 599...
✔ Saved 3 frames for asitwas_snap_down_start
Processing asitwas_snap_down at frame 599...
Processing asitwas_snap_down at frame 607...
Processing asitwas_snap_down at frame 616...
✔ Saved 3 frames for asitwas_snap_down
Processing asitwas_snap_down at frame 616...
Processing asitwas_snap_down at frame 625...
Processing asitwas_snap_down at frame 633...

In [ ]:
import pandas as pd
pd.Series(list(reference_data.keys())).value_counts().max()

1

In [ ]:
## Draw 1 Frame ##

import cv2
import json
import os
import joblib
import numpy as np

# Load timeline.json
with open("timeline.json", "r") as f:
    timeline_data = json.load(f)

# Extract moves
moves = timeline_data["moves"]

# Load reference data
reference_data = joblib.load("reference_moves/reference_data.pkl")

# Load the dance video
video_path = "videoplayback_25fps.mp4"  # Change this to your video file
cap = cv2.VideoCapture(video_path)

# Check if video loaded successfully
if not cap.isOpened():
    print("Error: Could not open video file.")
    exit()

fps = cap.get(cv2.CAP_PROP_FPS)  # Frames per second
print(f"Video FPS: {fps}")

# Define colors
WHITE = (255, 255, 255)
RED = (0, 0, 255)
GREEN = (0, 255, 0)
BLUE = (255, 0, 0)

# Joint connections based on the MediaPipe pose model
connections = [
    ("left_shoulder", "right_shoulder"),
    ("left_shoulder", "left_elbow"), ("left_elbow", "left_wrist"),
    ("right_shoulder", "right_elbow"), ("right_elbow", "right_wrist"),
    ("left_shoulder", "left_hip"), ("right_shoulder", "right_hip"),
    ("left_hip", "right_hip")
]

def draw_landmarks(frame, joints):
    """Draws landmarks and connections on the frame."""
    height, width, _ = frame.shape

    # Convert normalized joint coordinates to pixel values
    joint_positions = {key: (int(x * width), int(y * height)) for key, (x, y, z) in joints.items()}

    # Draw connections
    for joint1, joint2 in connections:
        if joint1 in joint_positions and joint2 in joint_positions:
            cv2.line(frame, joint_positions[joint1], joint_positions[joint2], BLUE, 2)

    # Draw joints
    for joint, (x, y) in joint_positions.items():
        cv2.circle(frame, (x, y), 5, RED, -1)

    return frame

# Process each move
for move in moves:
    move_name = move["name"]
    start_time = move["time"] + 2.1 # + 14.828
    duration = move["duration"]

    # Get the corresponding reference data
    if move_name not in reference_data:
        print(f"❌ No reference data found for {move_name}, skipping...")
        continue

    mid_time = start_time + (duration / 2)  # Middle of the move
    frame_idx = int(mid_time * fps)  # Convert to frame index

    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)  # Seek to the frame
    ret, frame = cap.read()
    
    if not ret:
        print(f"❌ Failed to capture frame for {move_name} at frame {frame_idx}!")
        continue

    print(f"🎨 Drawing landmarks for {move_name} at frame {frame_idx}...")

    joints = reference_data[move_name]["joints"]

    # Draw landmarks
    frame = draw_landmarks(frame, joints)

    # Display the frame
    cv2.imshow("Pose Landmarks", frame)
    key = cv2.waitKey(3000)  # Display for 500ms

    # Press 'q' to exit early
    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Video FPS: 25.0
🎨 Drawing landmarks for asitwas_jacket at frame 177...
🎨 Drawing landmarks for asitwas_snap at frame 194...
🎨 Drawing landmarks for asitwas_arm_up at frame 214...
🎨 Drawing landmarks for asitwas_snap_down_start at frame 272...
🎨 Drawing landmarks for asitwas_snap_down at frame 289...
🎨 Drawing landmarks for asitwas_snap_down at frame 306...
🎨 Drawing landmarks for asitwas_snap_down at frame 324...
🎨 Drawing landmarks for asitwas_snap_down_start_2 at frame 341...
🎨 Drawing landmarks for asitwas_snap_down at frame 358...
🎨 Drawing landmarks for asitwas_snap_down at frame 375...


In [ ]:
## Draw 3 Frames ##

import cv2
import json
import os
import joblib
import numpy as np

# Load timeline.json
with open("timeline.json", "r") as f:
    timeline_data = json.load(f)

# Extract moves
moves = timeline_data["moves"]

# Load reference data
reference_data = joblib.load("reference_moves_three/reference_data.pkl")

# Load the dance video
video_path = "videoplayback_25fps.mp4"  # Change this to your video file
cap = cv2.VideoCapture(video_path)

# Check if video loaded successfully
if not cap.isOpened():
    print("Error: Could not open video file.")
    exit()

fps = cap.get(cv2.CAP_PROP_FPS)  # Frames per second
print(f"Video FPS: {fps}")

# Define colors
WHITE = (255, 255, 255)
RED = (0, 0, 255)
GREEN = (0, 255, 0)
BLUE = (255, 0, 0)

# Joint connections based on the MediaPipe pose model
connections = [
    ("left_shoulder", "right_shoulder"),
    ("left_shoulder", "left_elbow"), ("left_elbow", "left_wrist"),
    ("right_shoulder", "right_elbow"), ("right_elbow", "right_wrist"),
    ("left_shoulder", "left_hip"), ("right_shoulder", "right_hip"),
    ("left_hip", "right_hip")
]

def draw_landmarks(frame, joints):
    """Draws landmarks and connections on the frame."""
    height, width, _ = frame.shape

    # Convert normalized joint coordinates to pixel values
    joint_positions = {key: (int(x * width), int(y * height)) for key, (x, y, z) in joints.items()}

    # Draw connections
    for joint1, joint2 in connections:
        if joint1 in joint_positions and joint2 in joint_positions:
            cv2.line(frame, joint_positions[joint1], joint_positions[joint2], BLUE, 2)

    # Draw joints
    for joint, (x, y) in joint_positions.items():
        cv2.circle(frame, (x, y), 5, RED, -1)

    return frame

# Process each move
for move in moves:
    move_name = move["name"]
    start_time = move["time"] + 2.1
    duration = move["duration"]

    # Get the corresponding reference data
    if move_name not in reference_data or len(reference_data[move_name]) < 3:
        print(f"❌ No reference data found for {move_name}, skipping...")
        continue

    frame_times = [
        start_time + (duration * 0.25),  # First quarter
        start_time + (duration * 0.5),   # Midpoint
        start_time + (duration * 0.75)   # Three-quarters
    ]
    frame_indices = [int(t * fps) for t in frame_times]  # Convert times to frame indices

    for i, frame_idx in enumerate(frame_indices):
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)  # Seek to the frame
        ret, frame = cap.read()
        
        if not ret:
            print(f"❌ Failed to capture frame {i+1} for {move_name} at frame {frame_idx}!")
            continue

        print(f"🎨 Drawing landmarks for {move_name} (Frame {i+1}) at frame {frame_idx}...")

        joints = reference_data[move_name][i]["joints"]  # Get the corresponding pose data

        # Draw landmarks
        frame = draw_landmarks(frame, joints)

        # Display the frame
        cv2.imshow("Pose Landmarks", frame)
        key = cv2.waitKey(1000)  # Display each frame for 1 second

        # Press 'q' to exit early
        if key == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


Video FPS: 25.0
🎨 Drawing landmarks for asitwas_jacket (Frame 1) at frame 173...
🎨 Drawing landmarks for asitwas_jacket (Frame 2) at frame 177...
🎨 Drawing landmarks for asitwas_jacket (Frame 3) at frame 181...
🎨 Drawing landmarks for asitwas_snap (Frame 1) at frame 190...
🎨 Drawing landmarks for asitwas_snap (Frame 2) at frame 194...
🎨 Drawing landmarks for asitwas_snap (Frame 3) at frame 199...
🎨 Drawing landmarks for asitwas_arm_up (Frame 1) at frame 208...
🎨 Drawing landmarks for asitwas_arm_up (Frame 2) at frame 214...
🎨 Drawing landmarks for asitwas_arm_up (Frame 3) at frame 219...
🎨 Drawing landmarks for asitwas_snap_down_start (Frame 1) at frame 268...
🎨 Drawing landmarks for asitwas_snap_down_start (Frame 2) at frame 272...
🎨 Drawing landmarks for asitwas_snap_down_start (Frame 3) at frame 276...
🎨 Drawing landmarks for asitwas_snap_down (Frame 1) at frame 285...
🎨 Drawing landmarks for asitwas_snap_down (Frame 2) at frame 289...
🎨 Drawing landmarks for asitwas_snap_down (Fram

In [ ]:
## Score 1 Frame ##

import cv2
import mediapipe as mp
import json
import numpy as np
import joblib

# Load timeline.json
with open("timeline.json", "r") as f:
    timeline_data = json.load(f)

moves = timeline_data["moves"]  # Extract move data

# Load reference move data
reference_data = joblib.load("reference_moves/reference_data.pkl")

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose()

# Load the dance video
video_path = "videoplayback_25fps.mp4"  # Change this to your video file
cap = cv2.VideoCapture(video_path)

# Check if video loaded successfully
if not cap.isOpened():
    print("Error: Could not open video file.")
    exit()

fps = cap.get(cv2.CAP_PROP_FPS)  # Frames per second
print(f"Video FPS: {fps}")

def calculate_angle(a, b, c):
    """Calculate angle between three 3D points using the arctangent function."""
    ba = np.array(a) - np.array(b)
    bc = np.array(c) - np.array(b)

    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))

    return np.degrees(angle)

def process_frame(image):
    """Extract pose landmarks and compute angles from a frame."""
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark

        # Joint indices for key body parts
        joint_ids = {
            "nose": 0,
            "left_shoulder": 11, "right_shoulder": 12,
            "left_elbow": 13, "right_elbow": 14,
            "left_wrist": 15, "right_wrist": 16,
            "left_hip": 23, "right_hip": 24
        }

        joints = {name: (landmarks[idx].x, landmarks[idx].y, landmarks[idx].z) for name, idx in joint_ids.items()}

        # Compute angles
        angles = {}

        if all(k in joints for k in ["left_shoulder", "left_elbow", "left_wrist"]):
            angles["left_elbow"] = calculate_angle(joints["left_shoulder"], joints["left_elbow"], joints["left_wrist"])
        if all(k in joints for k in ["right_shoulder", "right_elbow", "right_wrist"]):
            angles["right_elbow"] = calculate_angle(joints["right_shoulder"], joints["right_elbow"], joints["right_wrist"])
        if all(k in joints for k in ["left_elbow", "left_shoulder", "left_hip"]):
            angles["left_arm_lift"] = calculate_angle(joints["left_elbow"], joints["left_shoulder"], joints["left_hip"])
        if all(k in joints for k in ["right_elbow", "right_shoulder", "right_hip"]):
            angles["right_arm_lift"] = calculate_angle(joints["right_elbow"], joints["right_shoulder"], joints["right_hip"])

        return {"joints": joints, "angles": angles}

    return None

def score_angles(reference_angles, detected_angles):
    """Compare angles with reference and return a score."""
    total_score = 0
    count = 0

    for key in reference_angles:
        if key in detected_angles:
            diff = abs(reference_angles[key] - detected_angles[key])

            if diff < 20:
                score = 100  # Perfect
            elif diff < 30:
                score = 70  # Okay
            else:
                score = 40  # Bad

            total_score += score
            count += 1

    return total_score / count if count > 0 else 0  # Average score

# Initialize final accuracy tracking
total_moves = 0
total_accuracy = 0

# Process each move in the timeline
for move in moves:
    move_name = move["name"]
    start_time = move["time"] + 2.1 # + 14.828
    duration = move["duration"]

    mid_time = start_time + (duration / 2)  # Middle of the move
    frame_idx = int(mid_time * fps)  # Convert to frame index
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)  # Seek to the frame

    ret, frame = cap.read()
    if not ret:
        print(f"Failed to capture frame for {move_name} at time {mid_time}s (frame {frame_idx})!")
        continue

    print(f"Processing {move_name} at frame {frame_idx}...")

    detected_pose = process_frame(frame)
    if not detected_pose:
        print(f"❌ No pose detected for {move_name}")
        continue

    # Get reference pose angles
    if move_name in reference_data:
        reference_angles = reference_data[move_name]["angles"]
    else:
        print(f"❌ No reference data for {move_name}")
        continue

    # Score the move
    score = score_angles(reference_angles, detected_pose["angles"])
    total_moves += 1
    total_accuracy += score

    # Draw landmarks and score on the frame
    mp_drawing.draw_landmarks(frame, pose.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)).pose_landmarks, mp_pose.POSE_CONNECTIONS)

    # Display move name and score on frame
    cv2.putText(frame, f"Move: {move_name}", (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
    cv2.putText(frame, f"Score: {int(score)}", (30, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

    cv2.imshow(f"Dance Scoring", frame)
    key = cv2.waitKey(500)  # Display for 500ms

    # Press 'q' to exit early
    if key == ord('q'):
        break


# Print final accuracy
if total_moves > 0:
    final_accuracy = total_accuracy / total_moves
    print(f"\n✅ Final Accuracy: {final_accuracy:.2f}%")
else:
    print("\n❌ No valid moves were detected.")

cv2.destroyAllWindows()
cap.release()

Video FPS: 25.0
Processing asitwas_jacket at frame 177...
Processing asitwas_snap at frame 194...
Processing asitwas_arm_up at frame 214...
Processing asitwas_snap_down_start at frame 272...
Processing asitwas_snap_down at frame 289...
Processing asitwas_snap_down at frame 306...
Processing asitwas_snap_down at frame 324...
Processing asitwas_snap_down_start_2 at frame 341...
Processing asitwas_snap_down at frame 358...
Processing asitwas_snap_down at frame 375...
Processing asitwas_snap_down_end at frame 393...
Processing asitwas_arm_down at frame 410...
Processing asitwas_sun at frame 427...
Processing asitwas_fists at frame 444...
Processing asitwas_arm_down at frame 479...
Processing asitwas_sun at frame 496...
Processing asitwas_fists at frame 513...
Processing asitwas_circle at frame 548...
Processing asitwas_circle_2 at frame 582...
Processing asitwas_circle at frame 617...
Processing asitwas_circle_2 at frame 651...
Processing asitwas_arm_down at frame 686...
Processing asitwas

In [ ]:
## Score 3 Frames without drawing ref ##

import cv2
import mediapipe as mp
import json
import numpy as np
import joblib

# Load timeline.json
with open("timeline.json", "r") as f:
    timeline_data = json.load(f)

moves = timeline_data["moves"]  # Extract move data

# Load reference move data
reference_data = joblib.load("reference_moves_three/reference_data.pkl")

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose()

# Load the dance video
video_path = "videoplayback_25fps.mp4"  # Change this to your video file
cap = cv2.VideoCapture(video_path)

# Check if video loaded successfully
if not cap.isOpened():
    print("Error: Could not open video file.")
    exit()

fps = cap.get(cv2.CAP_PROP_FPS)  # Frames per second
print(f"Video FPS: {fps}")

def calculate_angle(a, b, c):
    """Calculate angle between three 3D points using the arctangent function."""
    ba = np.array(a) - np.array(b)
    bc = np.array(c) - np.array(b)

    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))

    return np.degrees(angle)

def process_frame(image):
    """Extract pose landmarks and compute angles from a frame."""
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark

        # Joint indices for key body parts
        joint_ids = {
            "nose": 0,
            "left_shoulder": 11, "right_shoulder": 12,
            "left_elbow": 13, "right_elbow": 14,
            "left_wrist": 15, "right_wrist": 16,
            "left_hip": 23, "right_hip": 24
        }

        joints = {name: (landmarks[idx].x, landmarks[idx].y, landmarks[idx].z) for name, idx in joint_ids.items()}

        # Compute angles
        angles = {}

        if all(k in joints for k in ["left_shoulder", "left_elbow", "left_wrist"]):
            angles["left_elbow"] = calculate_angle(joints["left_shoulder"], joints["left_elbow"], joints["left_wrist"])
        if all(k in joints for k in ["right_shoulder", "right_elbow", "right_wrist"]):
            angles["right_elbow"] = calculate_angle(joints["right_shoulder"], joints["right_elbow"], joints["right_wrist"])
        if all(k in joints for k in ["left_elbow", "left_shoulder", "left_hip"]):
            angles["left_arm_lift"] = calculate_angle(joints["left_elbow"], joints["left_shoulder"], joints["left_hip"])
        if all(k in joints for k in ["right_elbow", "right_shoulder", "right_hip"]):
            angles["right_arm_lift"] = calculate_angle(joints["right_elbow"], joints["right_shoulder"], joints["right_hip"])

        return {"joints": joints, "angles": angles}

    return None

def score_angles(reference_angles, detected_angles):
    """Compare angles with reference and return a score."""
    total_score = 0
    count = 0

    for key in reference_angles:
        if key in detected_angles:
            diff = abs(reference_angles[key] - detected_angles[key])

            if diff < 20:
                score = 100  # Perfect
            elif diff < 40:
                score = 70  # Okay
            else:
                score = 40  # Bad

            total_score += score
            count += 1

    return total_score / count if count > 0 else 0  # Average score

# Initialize final accuracy tracking
total_moves = 0
total_accuracy = 0

# Process each move in the timeline
for move in moves:
    move_name = move["name"]
    start_time = move["time"] + 2.1  # Adjust timing offset
    duration = move["duration"]

    # Get the three frame times (25%, 50%, and 75% of move duration)
    frame_times = [
        start_time + (duration * 0.25),  # First quarter
        start_time + (duration * 0.5),   # Midpoint
        start_time + (duration * 0.75)   # Three-quarters
    ]
    frame_indices = [int(t * fps) for t in frame_times]  # Convert times to frame indices

    if move_name not in reference_data or len(reference_data[move_name]) < 3:
        print(f"❌ No reference data found for {move_name}, skipping...")
        continue

    total_move_score = 0  # Sum of scores for the three frames
    valid_frames = 0  # Count of successfully processed frames

    for i, frame_idx in enumerate(frame_indices):
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)  # Seek to the frame
        ret, frame = cap.read()

        if not ret:
            print(f"❌ Failed to capture frame {i+1} for {move_name} at frame {frame_idx}!")
            continue

        print(f"🎨 Processing {move_name} (Frame {i+1}) at frame {frame_idx}...")

        detected_pose = process_frame(frame)
        if not detected_pose:
            print(f"❌ No pose detected for {move_name} (Frame {i+1})")
            continue

        # Get reference pose angles for this frame
        reference_angles = reference_data[move_name][i]["angles"]

        # Score the move
        score = score_angles(reference_angles, detected_pose["angles"])
        total_move_score += score
        valid_frames += 1

        # Draw landmarks and score on the frame
        mp_drawing.draw_landmarks(frame, pose.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)).pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # Display move name and score on frame
        cv2.putText(frame, f"Move: {move_name}", (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
        cv2.putText(frame, f"Score: {int(score)}", (30, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

        cv2.imshow(f"Dance Scoring", frame)
        key = cv2.waitKey(100)  # Display for 1 second

        # Press 'q' to exit early
        if key == ord('q'):
            break

    if valid_frames > 0:
        move_final_score = total_move_score / valid_frames  # Average score across frames
        total_moves += 1
        total_accuracy += move_final_score

        print(f"✅ Final score for {move_name}: {move_final_score:.2f}%\n")

# Print final accuracy
if total_moves > 0:
    final_accuracy = total_accuracy / total_moves
    print(f"\n✅ Final Accuracy: {final_accuracy:.2f}%")
else:
    print("\n❌ No valid moves were detected.")

cv2.destroyAllWindows()
cap.release()

Video FPS: 25.0
🎨 Processing asitwas_jacket (Frame 1) at frame 173...
🎨 Processing asitwas_jacket (Frame 2) at frame 177...
🎨 Processing asitwas_jacket (Frame 3) at frame 181...
✅ Final score for asitwas_jacket: 87.50%

🎨 Processing asitwas_snap (Frame 1) at frame 190...
🎨 Processing asitwas_snap (Frame 2) at frame 194...
🎨 Processing asitwas_snap (Frame 3) at frame 199...
✅ Final score for asitwas_snap: 85.00%

🎨 Processing asitwas_arm_up (Frame 1) at frame 208...
🎨 Processing asitwas_arm_up (Frame 2) at frame 214...
🎨 Processing asitwas_arm_up (Frame 3) at frame 219...
✅ Final score for asitwas_arm_up: 87.50%

🎨 Processing asitwas_snap_down_start (Frame 1) at frame 268...
🎨 Processing asitwas_snap_down_start (Frame 2) at frame 272...
🎨 Processing asitwas_snap_down_start (Frame 3) at frame 276...
✅ Final score for asitwas_snap_down_start: 82.50%

🎨 Processing asitwas_snap_down (Frame 1) at frame 285...
🎨 Processing asitwas_snap_down (Frame 2) at frame 289...
🎨 Processing asitwas_snap_

In [47]:
## Score 3 Frames with drawing ref ##

import cv2
import mediapipe as mp
import json
import numpy as np
import joblib

# Load timeline.json
with open("timeline.json", "r") as f:
    timeline_data = json.load(f)

moves = timeline_data["moves"]  # Extract move data

# Load reference move data
reference_data = joblib.load("reference_moves_three/reference_data.pkl")

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose()

# Load the dance video
video_path = "videoplayback_25fps.mp4"  # Change this to your video file
cap = cv2.VideoCapture(video_path)

# Check if video loaded successfully
if not cap.isOpened():
    print("Error: Could not open video file.")
    exit()

fps = cap.get(cv2.CAP_PROP_FPS)  # Frames per second
print(f"Video FPS: {fps}")

# Define Colors
BLUE = (255, 0, 0)  # Reference Pose Color
RED = (0, 0, 255)   # Detected Pose Color

# Joint Connections
connections = [
    ("left_shoulder", "left_elbow"), ("left_elbow", "left_wrist"),
    ("right_shoulder", "right_elbow"), ("right_elbow", "right_wrist"),
    ("left_shoulder", "left_hip"), ("right_shoulder", "right_hip"),
]

def calculate_angle(a, b, c):
    """Calculate angle between three 3D points using the arctangent function."""
    ba = np.array(a) - np.array(b)
    bc = np.array(c) - np.array(b)

    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))

    return np.degrees(angle)

def process_frame(image):
    """Extract pose landmarks from a frame and compute only left elbow and left arm lift angles."""
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark

        # Extract key joint coordinates (normalized values)
        joint_ids = {
            "left_shoulder": 11,
            "left_elbow": 13,
            "left_wrist": 15,
            "left_hip": 23
        }

        joints = {name: (landmarks[idx].x, landmarks[idx].y, landmarks[idx].z) for name, idx in joint_ids.items()}

        # Compute angles
        angles = {}

        if all(k in joints for k in ["left_shoulder", "left_elbow", "left_wrist"]):
            angles["left_elbow"] = calculate_angle(joints["left_shoulder"], joints["left_elbow"], joints["left_wrist"])
        if all(k in joints for k in ["left_elbow", "left_shoulder", "left_hip"]):
            angles["left_arm_lift"] = calculate_angle(joints["left_elbow"], joints["left_shoulder"], joints["left_hip"])

        return {"joints": joints, "angles": angles}

    return None



def draw_landmarks(frame, joints, color):
    """Draws landmarks and connections on the frame."""
    height, width, _ = frame.shape

    # Convert normalized joint coordinates to pixel values
    joint_positions = {key: (int(x * width), int(y * height)) for key, (x, y, z) in joints.items()}

    # Draw connections
    for joint1, joint2 in connections:
        if joint1 in joint_positions and joint2 in joint_positions:
            cv2.line(frame, joint_positions[joint1], joint_positions[joint2], color, 2)

    # Draw joints
    for joint, (x, y) in joint_positions.items():
        cv2.circle(frame, (x, y), 5, color, -1)

def score_angles(reference_angles, detected_angles):
    """Compare angles with reference and return a score."""
    total_score = 0
    count = 0

    for key in reference_angles:
        if key in detected_angles:
            diff = abs(reference_angles[key] - detected_angles[key])

            if diff < 20:
                score = 100  # Perfect
            elif diff < 40:
                score = 70  # Okay
            else:
                score = 40  # Bad

            total_score += score
            count += 1

    return total_score / count if count > 0 else 0  # Average score

# Initialize final accuracy tracking
total_moves = 0
total_accuracy = 0

# Process each move in the timeline
for move in moves:
    move_name = move["name"]
    start_time = move["time"] + 2.1  # Adjust timing offset
    duration = move["duration"]

    # Get the three frame times (25%, 50%, and 75% of move duration)
    frame_times = [
        start_time + (duration * 0.25),
        start_time + (duration * 0.5),
        start_time + (duration * 0.75)
    ]
    frame_indices = [int(t * fps) for t in frame_times]  # Convert times to frame indices

    if move_name not in reference_data or len(reference_data[move_name]) < 3:
        print(f"❌ No reference data found for {move_name}, skipping...")
        continue

    total_move_score = 0  
    valid_frames = 0  

    for i, frame_idx in enumerate(frame_indices):
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)  
        ret, frame = cap.read()

        if not ret:
            print(f"❌ Failed to capture frame {i+1} for {move_name} at frame {frame_idx}!")
            continue

        print(f"🎨 Processing {move_name} (Frame {i+1}) at frame {frame_idx}...")

        detected_pose = process_frame(frame)
        if not detected_pose:
            print(f"❌ No pose detected for {move_name} (Frame {i+1})")
            continue

        reference_joints = reference_data[move_name][i]["joints"]
        reference_angles = reference_data[move_name][i]["angles"]

        score = score_angles(reference_angles, detected_pose["angles"])
        total_move_score += score
        valid_frames += 1

        draw_landmarks(frame, reference_joints, BLUE)  # Draw reference pose (blue)
        draw_landmarks(frame, detected_pose["joints"], RED)  # Draw detected pose (red)

        cv2.putText(frame, f"Move: {move_name}", (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
        cv2.putText(frame, f"Score: {int(score)}", (30, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

        cv2.imshow("Dance Scoring", frame)
        if cv2.waitKey(100) == ord('q'):
            break

    if valid_frames > 0:
        total_moves += 1
        total_accuracy += total_move_score / valid_frames

print(f"\n✅ Final Accuracy: {total_accuracy / total_moves:.2f}%") if total_moves else print("\n❌ No valid moves detected.")
cv2.destroyAllWindows()
cap.release()

Video FPS: 25.0
🎨 Processing asitwas_jacket (Frame 1) at frame 173...
🎨 Processing asitwas_jacket (Frame 2) at frame 177...
🎨 Processing asitwas_jacket (Frame 3) at frame 181...
🎨 Processing asitwas_snap (Frame 1) at frame 190...
🎨 Processing asitwas_snap (Frame 2) at frame 194...
🎨 Processing asitwas_snap (Frame 3) at frame 199...
🎨 Processing asitwas_arm_up (Frame 1) at frame 208...
🎨 Processing asitwas_arm_up (Frame 2) at frame 214...
🎨 Processing asitwas_arm_up (Frame 3) at frame 219...
🎨 Processing asitwas_snap_down_start (Frame 1) at frame 268...
🎨 Processing asitwas_snap_down_start (Frame 2) at frame 272...
🎨 Processing asitwas_snap_down_start (Frame 3) at frame 276...
🎨 Processing asitwas_snap_down (Frame 1) at frame 285...
🎨 Processing asitwas_snap_down (Frame 2) at frame 289...
🎨 Processing asitwas_snap_down (Frame 3) at frame 293...
🎨 Processing asitwas_snap_down (Frame 1) at frame 302...
🎨 Processing asitwas_snap_down (Frame 2) at frame 306...
🎨 Processing asitwas_snap_down

In [ ]:
7 - 4.65517139

2.3448286100000004